In [1]:
import os
from core.slack import SlackMessageRetriever
from core.bigquery import BigqueryProcessor


slack_app = SlackMessageRetriever(env_name="SLACK_TOKEN_BOT")
bigquery_client = BigqueryProcessor(
    env_name="GOOGLE_APPLICATION_CREDENTIALS", database_id="geultto_9th"
)
user_pd = bigquery_client.run_query_to_dataframe(
    query="""
    SELECT SUM(cnt_emoji) AS total_emoji
        , SUM(cnt_post) AS total_post
        , SUM(cnt_thread) AS total_thread     
    FROM (
    SELECT users.*
        , cnt_emoji 
        , cnt_post
        , cnt_thread
    FROM `geultto.geultto_9th.users` AS users
    LEFT JOIN (
        SELECT '이모지수' AS group_name
            , user_id
            , COUNT(*) cnt_emoji
        FROM (
        SELECT JSON_EXTRACT_SCALAR(json, '$.name') AS name
            , REPLACE(user_id, '"', "") AS user_id
        FROM `geultto.geultto_9th.slack_conversation_master`,
        UNNEST(JSON_EXTRACT_ARRAY(reactions)) AS json,
        UNNEST(JSON_EXTRACT_ARRAY(json, '$.user_id')) AS user_id 
        WHERE tddate <= '2023-12-24'
        ) A
        GROUP BY user_id
    ) emoji_cnt
    ON users.user_id = emoji_cnt.user_id
    LEFT JOIN (
        SELECT user_id
            , COUNT(*) cnt_post
        FROM `geultto.geultto_9th.slack_conversation_master`
        WHERE tddate <= '2023-12-24'
        AND message_type = 'post'
        AND (text not like ('%님이 채널에 참여함%') AND text not like ('%integration to this channel%'))
        GROUP BY user_id
    ) post_cnt
    ON users.user_id = post_cnt.user_id
    LEFT JOIN (
        SELECT user_id
            , COUNT(*) cnt_thread
        FROM `geultto.geultto_9th.slack_conversation_master`
        WHERE tddate <= '2023-12-24'
        AND message_type = 'thread'
        GROUP BY user_id  
    ) thread_cnt
    ON users.user_id = thread_cnt.user_id
    ) A
    """
)

user_pd.head()

,total_emoji,total_post,total_thread
0,82791,3712,7570


In [3]:
user_pd['total_emoji'][0]

82791

In [2]:
# for _, row in user_pd.loc[(user_pd['name'].isin(['이승환', '김유경']))].head().iterrows():
#     print(row[0], row[1], row[2], row[3], row[4], row[5])

In [27]:
def process_row(row):
    text = row['text'][:80]  # 텍스트의 첫 30자
    link = row['permalink']
    emoji_count = row['total_emoji_count']
    return f"""텍스트: {text}..., 
게시글링크: {link}, 
총 이모지 개수: {emoji_count}"""

In [28]:
slack_messages = [process_row(row) for index, row in user_pd.iterrows()]

In [29]:
slack_message = "\n\n".join(slack_messages)
print(slack_message)

텍스트: &gt; :sunflower: *외향적인 해바라기* 님이 *2023. 12. 11. 오후 12:47:17* 에 등록한 메시지입니다.  비도 오고..., 
게시글링크: https://geultto9.slack.com/archives/C066SQT5MBK/p1702266437425269, 
총 이모지 개수: 61

텍스트: &gt; :octopus: *유순한 문어* 님이 *2023. 12. 15. 오후 12:26:34* 에 등록한 메시지입니다.  정말 대나무 숲에 ..., 
게시글링크: https://geultto9.slack.com/archives/C066SQT5MBK/p1702610794259859?thread_ts=1702610794.259859&cid=C066SQT5MBK, 
총 이모지 개수: 50

텍스트: @까다로운 원숭이

다른 직업 고민해보시죠라는 댓글은 폭력적인 댓글 같아요. 제멋대로 꿀벌님의 메시지의 핵심은 "코테를 못봐서 속상하다. 위로를..., 
게시글링크: https://geultto9.slack.com/archives/C066SQT5MBK/p1702795770565269?thread_ts=1702725481.766149&cid=C066SQT5MBK, 
총 이모지 개수: 41

텍스트: &gt; :butterfly: *미소짓는 나비* 님이 *2023. 12. 17. 오후 4:04:49* 에 등록한 메시지입니다.  헤어졌어요. 정..., 
게시글링크: https://geultto9.slack.com/archives/C066SQT5MBK/p1702796689485639?thread_ts=1702796689.485639&cid=C066SQT5MBK, 
총 이모지 개수: 36

텍스트: &gt; :wolf: *격노한 늑대* 님이 *2023. 12. 18. 오후 3:31:42* 에 등록한 메시지입니다.  안녕하세요. 현재 4년 차..., 
게시글링크: https://geultto9.slack.com/archives/C066SQT5MBK/p1

In [30]:
for i in range(0, 10):
    if i == 2:
        continue
    print(i)

0
1
3
4
5
6
7
8
9


# 라이브러리 세팅

In [1]:
import os
from typing import List, Dict
import json
from datetime import datetime
import ssl
import certifi
import time

import pandas as pd
from slack_sdk import WebClient
from slack_bolt import App

from datetime import datetime
from dateutil.relativedelta import relativedelta

from core.date_utils import get_daily_datelist

# 슬랙앱 & 날짜 함수 세팅

In [2]:
token = os.environ.get("SLACK_TOKEN_USER")
ssl_context = ssl.create_default_context(cafile=certifi.where())
app = App(client=WebClient(token=token, ssl=ssl_context))

In [3]:
def timestamp_to_unixtime(sdatetime: datetime) -> list:
    sdatetime_minus1 = sdatetime + relativedelta(days=-1)
    start_unixtime = time.mktime((sdatetime_minus1).timetuple())
    end_unixtime = time.mktime(sdatetime.timetuple()) - 1e-6  # 23:59:59 99999 까지
    return [start_unixtime, end_unixtime]

In [4]:
test_date = get_daily_datelist(start_date="2024-01-11", end_date="2024-01-11")
print(test_date)

[datetime.datetime(2024, 1, 11, 0, 0)]


In [5]:
unixtime_set = timestamp_to_unixtime(sdatetime=test_date[0])
print(unixtime_set)

[1704812400.0, 1704898799.999999]


# post & thread 세팅하기

In [34]:
channel_id = 'C066AQH75EZ'

In [35]:
posts = app.client.conversations_history(
    channel=channel_id,
    latest=unixtime_set[1],
    oldest=unixtime_set[0],
)

In [46]:
target_post = posts['messages']

dict_keys(['type', 'subtype', 'text', 'ts', 'username', 'bot_id', 'attachments', 'reactions'])

In [47]:
'user' not in target_post[5].keys()

True

In [10]:
target_ts = '1700403928.943909'

In [11]:
# threads = app.client.conversations_replies(channel=channel_id, ts=target_ts)['messages']
# threads[1]

In [12]:
# thread_ts = 1701005898.099139

# replies 테스트하기

In [16]:
# post테스트 완료
reactions_post = app.client.reactions_get(channel=channel_id, timestamp=target_ts, full=True)['message']
len(reactions_post['reactions'][0]['users'])


102

In [20]:
reactions_post['reactions']

[{'name': 'wave',
  'users': ['U066GPP3G20',
   'U066DKSRHQT',
   'U066E7MDTNF',
   'U066G6JPL3E',
   'U066T00QN65',
   'U0673AGGK5W',
   'U066GPLUQLC',
   'U066T053D8R',
   'U0667MM8WMC',
   'U066BD3DN2Z',
   'U0673ANED16',
   'U0667MSP9JS',
   'U066E899M8T',
   'U066SBYNGE5',
   'U066AQSBRPF',
   'U065ZR8FXAT',
   'U066BDJJF7F',
   'U066E8RBDUK',
   'U06673WKMHU',
   'U066BDY5761',
   'U066BDWJKRT',
   'U066GRCCLRJ',
   'U066SV8A5JM',
   'U066BEE9NCV',
   'U065ZQAUAMV',
   'U066T1LT7DX',
   'U066E9SAK4K',
   'U066GS3A04C',
   'U066T217U2V',
   'U065ZSLK423',
   'U0673C9QQ3S',
   'U0667RQ643Y',
   'U066E7U9QFM',
   'U06673WCJP8',
   'U06683Y8XRU',
   'U066H9M177E',
   'U066EPLNNM9',
   'U06687X2E4W',
   'U066AQRV0E9',
   'U0672NB1EN4',
   'U066ET8CMFC',
   'U0673SAU2L8',
   'U0673V7JAAC',
   'U066EV6EKJN',
   'U066GPMRG9J',
   'U0660FU6MRD',
   'U066TKHCF4Z',
   'U0660GKF2TH',
   'U06742Z2P6U',
   'U066T0TBX6D',
   'U0674490K40',
   'U066EUM5U0K',
   'U0660JWQ9RD',
   'U066SV1JDU1',
 

In [57]:
# post테스트 완료
reactions_thread = app.reactions_get(channel=channel_id, timestamp=thread_ts, full=True)['message']
reactions_thread


{'client_msg_id': '95d3f704-36bf-4779-bd68-114e4d894da1',
 'type': 'message',
 'text': '<@U06790T7S2E> 이모지 확인해줘',
 'user': 'U0666UTKF8W',
 'ts': '1701005898.099139',
 'blocks': [{'type': 'rich_text',
   'block_id': 'zpwVA',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'user', 'user_id': 'U06790T7S2E'},
      {'type': 'text', 'text': ' 이모지 확인해줘'}]}]}],
 'team': 'T066AL99EE9',
 'thread_ts': '1701003194.380469',
 'parent_user_id': 'U0666UTKF8W',
 'reactions': [{'name': 'snail',
   'users': ['U066HRY5AF6',
    'U066UGGJGTT',
    'U067Q2P1AN4',
    'U066SV3JQE5',
    'U0667RQ643Y',
    'U0676062H6C',
    'U066WMSAJLR',
    'U066BCCJCAH',
    'U066SV1JDU1',
    'U066T053D8R',
    'U0660RDQAGP',
    'U06745JC3TJ',
    'U066GQ0UNA0',
    'U065ZQAUAMV',
    'U066W0MJDLZ',
    'U066CC2M38D',
    'U066SV2BHCH',
    'U066GR3NS76',
    'U067233613M',
    'U0673SAU2L8',
    'U0667LKJ4R4',
    'U067430ASAC',
    'U066EATSX6F',
    'U066W72QJRF',
    'U066T0NBEG1',
    'U06

# 링크 테스트

In [71]:
get_link = app.client.chat_getPermalink(channel=channel_id, message_ts=target_ts)
get_link['permalink']

'https://geultto9.slack.com/archives/C0667GPQ1F0/p1701003194380469?thread_ts=1701003194.380469&cid=C0667GPQ1F0'